In [ ]:
import os
# suppress TensorFlow GPU plugin registration warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

import pandas as pd
import numpy as np
import time, resource
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# 1) Load CSV
train_path = '/kaggle/input/uci-har-dataset/train.csv'
test_path  = '/kaggle/input/uci-har-dataset/test.csv'
df_train = pd.read_csv(train_path)
df_test  = pd.read_csv(test_path)

# 2) Features & labels
X_train_raw = df_train.drop("activity", axis=1).values
y_train_raw = df_train["activity"].values
X_test_raw  = df_test.drop("activity", axis=1).values
y_test_raw  = df_test["activity"].values

# 3) Encode labels → integers (zero-indexed) → one-hot
le = LabelEncoder()
y_train_int = le.fit_transform(y_train_raw)  # now 0…5
y_test_int  = le.transform(y_test_raw)
y_train = to_categorical(y_train_int)
y_test  = to_categorical(y_test_int)
n_classes = y_train.shape[1]
class_names = [str(i) for i in range(n_classes)]

# 4) Scale features & reshape for CNN
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_raw).reshape(-1, X_train_raw.shape[1], 1)
X_test  = scaler.transform(X_test_raw).reshape(-1, X_test_raw.shape[1], 1)

# 5) Model builder
def build_model(lr=1e-3, activation='relu', num_conv_blocks=2):
    model = Sequential()
    for i in range(num_conv_blocks):
        filters = 64 * (2**i)
        if i == 0:
            model.add(Conv1D(filters, 3, activation=activation,
                             input_shape=(X_train.shape[1],1)))
        else:
            model.add(Conv1D(filters, 3, activation=activation))
        model.add(MaxPooling1D(2))
    model.add(Flatten())
    model.add(Dense(100, activation=activation))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=lr),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# 6) Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

# 7) Train & evaluate the base model
print("\n===== BASE MODEL =====")
base_params = {'lr':1e-3, 'activation':'relu', 'num_conv_blocks':2}
model = build_model(**base_params)
t0, mem0 = time.time(), resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
history = model.fit(
    X_train, y_train,
    epochs=50, batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)
loss, acc = model.evaluate(X_test, y_test, verbose=0)
y_pred = np.argmax(model.predict(X_test), axis=1)
t1, mem1 = time.time(), resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

print(f"\nBase → Test accuracy: {acc:.4f}")
print(f"⏱ Time: {t1-t0:.1f} sec | 💾 Memory: {(mem1-mem0)/1024:.1f} MB")
print("Classification Report:\n", classification_report(y_test_int, y_pred, target_names=class_names))
cm = confusion_matrix(y_test_int, y_pred)
print("Confusion Matrix:\n", cm)

# plot confusion matrix
plt.figure(figsize=(6,6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Base — Confusion Matrix")
ticks = np.arange(n_classes)
plt.xticks(ticks, class_names, rotation=45)
plt.yticks(ticks, class_names)
plt.ylabel("True")
plt.xlabel("Predicted")
plt.colorbar()
thresh = cm.max()/2
for i in range(n_classes):
    for j in range(n_classes):
        plt.text(j, i, format(cm[i,j], 'd'),
                 ha="center", va="center",
                 color="white" if cm[i,j]>thresh else "black")
plt.tight_layout()
plt.show()

# --- ADDED: learning curves for Base ---
plt.figure()
plt.plot(history.history['accuracy'],    label='train_acc')
plt.plot(history.history['val_accuracy'],label='val_acc')
plt.title("Base — Accuracy")
plt.xlabel("Epoch"); plt.ylabel("Accuracy")
plt.legend(); plt.show()

plt.figure()
plt.plot(history.history['loss'],    label='train_loss')
plt.plot(history.history['val_loss'],label='val_loss')
plt.title("Base — Loss")
plt.xlabel("Epoch"); plt.ylabel("Loss")
plt.legend(); plt.show()


# 8) Grid search (each hyperparameter separately)
grid_params = {
    'Learning_Rate': [
        {'lr':1e-4, 'activation':'relu', 'num_conv_blocks':2},
        {'lr':1e-3, 'activation':'relu', 'num_conv_blocks':2},
        {'lr':1e-2, 'activation':'relu', 'num_conv_blocks':2},
    ],
    'Activation_Function': [
        {'lr':1e-3, 'activation':'relu',   'num_conv_blocks':2},
        {'lr':1e-3, 'activation':'tanh',   'num_conv_blocks':2},
        {'lr':1e-3, 'activation':'sigmoid','num_conv_blocks':2},
    ],
    'Conv_Blocks': [
        {'lr':1e-3, 'activation':'relu', 'num_conv_blocks':1},
        {'lr':1e-3, 'activation':'relu', 'num_conv_blocks':2},
        {'lr':1e-3, 'activation':'relu', 'num_conv_blocks':3},
    ]
}

for category, configs in grid_params.items():
    print(f"\n\n===== TUNING {category} =====")
    for idx, params in enumerate(configs, 1):
        name = f"{category}_{idx}"
        print(f"\n--- CONFIG: {name} → {params} ---")
        model = build_model(**params)
        t0, mem0 = time.time(), resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
        history = model.fit(
            X_train, y_train,
            epochs=50, batch_size=64,
            validation_split=0.2,
            callbacks=[early_stop],
            verbose=1
        )
        loss, acc = model.evaluate(X_test, y_test, verbose=0)
        y_pred = np.argmax(model.predict(X_test), axis=1)
        t1, mem1 = time.time(), resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

        print(f"\n{name} → Test accuracy: {acc:.4f}")
        print(f"⏱ Time: {t1-t0:.1f} sec | 💾 Memory: {(mem1-mem0)/1024:.1f} MB")
        print("Classification Report:\n", classification_report(y_test_int, y_pred, target_names=class_names))
        cm = confusion_matrix(y_test_int, y_pred)
        print("Confusion Matrix:\n", cm)

        # plot confusion matrix
        plt.figure(figsize=(6,6))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title(f"{name} — Confusion Matrix")
        plt.xticks(ticks, class_names, rotation=45)
        plt.yticks(ticks, class_names)
        plt.ylabel("True"); plt.xlabel("Predicted")
        plt.colorbar()
        thresh = cm.max()/2
        for i in range(n_classes):
            for j in range(n_classes):
                plt.text(j, i, format(cm[i,j], 'd'),
                         ha="center", va="center",
                         color="white" if cm[i,j]>thresh else "black")
        plt.tight_layout()
        plt.show()

        # --- ADDED: learning curves for this config ---
        plt.figure()
        plt.plot(history.history['accuracy'],    label='train_acc')
        plt.plot(history.history['val_accuracy'],label='val_acc')
        plt.title(f"{name} — Accuracy")
        plt.xlabel("Epoch"); plt.ylabel("Accuracy")
        plt.legend(); plt.show()

        plt.figure()
        plt.plot(history.history['loss'],    label='train_loss')
        plt.plot(history.history['val_loss'],label='val_loss')
        plt.title(f"{name} — Loss")
        plt.xlabel("Epoch"); plt.ylabel("Loss")
        plt.legend(); plt.show()
